<a href="https://colab.research.google.com/github/Manaswi3104/Plant_Disease_Detection/blob/main/Soyaleaf_disease_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!rm -rf yolov5


/content


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Clone the YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

# Install required dependencies
!pip install -U -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17493, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 17493 (delta 1), reused 0 (delta 0), pack-reused 17490 (from 3)
Receiving objects: 100% (17493/17493), 16.59 MiB | 25.66 MiB/s, done.
Resolving deltas: 100% (11992/11992), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.4 MB/s eta 0:00:00

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sivm205/soybean-diseased-leaf-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/soybean-diseased-leaf-dataset


In [5]:
import os

# Replace 'path' with the actual variable name if needed
for root, dirs, files in os.walk(path):
    print("📁 Directory:", root)
    for name in files[:5]:  # print first 5 files per folder
        print("   📄", name)
    print("---")

📁 Directory: /kaggle/input/soybean-diseased-leaf-dataset
---
📁 Directory: /kaggle/input/soybean-diseased-leaf-dataset/ferrugen
   📄 DSC_0025.jpg
   📄 DSC_0008.jpg
   📄 DSC_0016.jpg
   📄 DSC_0019.jpg
   📄 DSC_0148.jpg
---
📁 Directory: /kaggle/input/soybean-diseased-leaf-dataset/Yellow Mosaic
   📄 YM (110).jpg
   📄 YM (77).jpg
   📄 YM (78).jpg
   📄 YM (95).jpg
   📄 YM (27).jpg
---
📁 Directory: /kaggle/input/soybean-diseased-leaf-dataset/brown_spot
   📄 bs (24).bmp
   📄 bs (10).JPG
   📄 bs (25).bmp
   📄 bs (47).bmp
   📄 bs (26).JPG
---
📁 Directory: /kaggle/input/soybean-diseased-leaf-dataset/powdery_mildew
   📄 aug_aug_DSC_0071.jpg
   📄 aug_DSC_0231.jpg
   📄 aug_aug_DSC_0085.jpg
   📄 aug_DSC_0075.jpg
   📄 aug_aug_DSC_0062.jpg
---
📁 Directory: /kaggle/input/soybean-diseased-leaf-dataset/septoria
   📄 DSC_0080.jpg
   📄 Septoriaf.jpg
   📄 DSC_0091.jpg
   📄 DSC_0090.jpg
   📄 DSC_0094.jpg
---
📁 Directory: /kaggle/input/soybean-diseased-leaf-dataset/Southern blight
   📄 DSC_0099.jpg
   📄 DSC_01

In [6]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load dataset
data_path = '/kaggle/input/soybean-diseased-leaf-dataset'
dataset = datasets.ImageFolder(root=data_path, transform=transform)

# Split into train and test sets, and load into DataLoader
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
!pip install split-folders

In [8]:
import splitfolders

splitfolders.ratio(
    input='/kaggle/input/soybean-diseased-leaf-dataset',
    output='/content/yolov5/soybean_cls_split',
    seed=42,
    ratio=(.8, .2)  # 80% train, 20% val
)


Copying files: 701 files [00:29, 24.11 files/s] 


In [10]:
%cd /content/yolov5

!python classify/train.py \
  --model yolov5s-cls.pt \
  --data soybean_cls_split \
  --epochs 20 \
  --img 224 \
  --batch 32

/content/yolov5
/content/yolov5/utils/general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-06-17 09:30:52.731606: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750152653.069198    4506 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750152653.162023    4506 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for pl

In [11]:
!python classify/val.py \
  --weights runs/train-cls/exp/weights/best.pt \
  --data soybean_cls_split \
  --imgsz 224 \
  --batch-size 128 \
  --verbose



/content/yolov5/utils/general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
classify/val: data=soybean_cls_split, weights=['runs/train-cls/exp/weights/best.pt'], batch_size=128, imgsz=224, device=, workers=8, verbose=True, project=runs/val-cls, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.13 torch-2.7.1+cu126 CPU

Fusing layers... 
Model summary: 117 layers, 4179498 parameters, 0 gradients, 10.4 GFLOPs
validating:   0% 0/2 [00:00<?, ?it/s]/content/yolov5/classify/val.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=device.type != "cpu"):
validating: 100% 2/2 [00:29<00:00, 14.91s/it]
             

In [13]:
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source /content/image.jpeg


/content/yolov5/utils/general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
classify/predict: weights=['runs/train-cls/exp/weights/best.pt'], source=/content/image.jpeg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.13 torch-2.7.1+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 117 layers, 4179498 parameters, 0 gradients, 10.4 GFLOPs
100% 755k/755k [00:00<00:00, 21.3MB/s]
image 1/1 /content/image.jpeg: 224x224 brown_spot 0.91, septoria 0.01, Southern blight 0.01, Yellow Mosaic 0.01, Mossaic Virus 0.01, 3.0ms
Speed: 0

In [15]:
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source /content/image3.png

/content/yolov5/utils/general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
classify/predict: weights=['runs/train-cls/exp/weights/best.pt'], source=/content/image3.png, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.13 torch-2.7.1+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 117 layers, 4179498 parameters, 0 gradients, 10.4 GFLOPs
image 1/1 /content/image3.png: 224x224 Yellow Mosaic 0.87, bacterial_blight 0.02, septoria 0.02, Mossaic Virus 0.02, ferrugen 0.02, 4.9ms
Speed: 0.4ms pre-process, 4.9ms inference, 21.9m

In [14]:
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source /content/image2.jpeg

/content/yolov5/utils/general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
classify/predict: weights=['runs/train-cls/exp/weights/best.pt'], source=/content/image2.jpeg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.13 torch-2.7.1+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 117 layers, 4179498 parameters, 0 gradients, 10.4 GFLOPs
image 1/1 /content/image2.jpeg: 224x224 Sudden Death Syndrone 0.92, ferrugen 0.01, crestamento 0.01, septoria 0.01, brown_spot 0.01, 3.0ms
Speed: 0.3ms pre-process, 3.0ms inference, 13.

In [26]:
# If in Google Colab
from google.colab import files
files.download('runs/train-cls/exp/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [14]:
!pip install pyngrok


In [15]:
!pip install pillow

In [16]:
import importlib
import subprocess
import sys

def install_if_missing(package):
    try:
        importlib.import_module(package)
        print(f"{package} is already installed ✅")
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Check and install each package
install_if_missing("streamlit")
install_if_missing("pyngrok")
install_if_missing("torch")
install_if_missing("torchvision")


streamlit is already installed ✅
pyngrok is already installed ✅
torch is already installed ✅
torchvision is already installed ✅


In [17]:
!curl ipv4.icanhazip.com

34.19.18.200


In [18]:
!ngrok config add-authtoken 2yZp8YUfuFWKVRJ6TiOkl84nvVa_5MQeNrFd95Preq7xtVHex


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [24]:
!pkill -f streamlit
!killall ngrok


In [22]:
from torchvision.datasets import ImageFolder
dataset = ImageFolder("/content/yolov5/soybean_cls_split")
print(dataset.classes)

['train', 'val']


In [25]:
from pyngrok import ngrok
import threading
import time
import os

# Kill old streamlit/ngrok if running
os.system("pkill -f streamlit")
os.system("killall ngrok")

def run():
    os.system("streamlit run app.py")

thread = threading.Thread(target=run)
thread.start()
time.sleep(5)

# Optional: set auth token (only once per Colab session)
# ngrok.set_auth_token("your_token_here")

public_url = ngrok.connect(8501)
print("App running at:", public_url)


App running at: NgrokTunnel: "https://b52e-34-19-18-200.ngrok-free.app" -> "http://localhost:8501"
